# Notebook - Análise de Vendas

Este notebook dá continuidade ao processo iniciado nos scripts:

- `src/generate_dataset.py` > Geração de dados sintéticos de vendas  ;
- `src/clean_dataset.py` > Limpeza básica do dataset  ;

## Objetivos neste notebook:
1. Calcular o **total de vendas** por produto;
2. Identificar o **produto com maior número de vendas totais**;
3. Realizar **análises exploratórias**;
4. Criar **visualizações gráficas** para identificar padrões.

Ao final, serão apresentados **insights** extraídos dos dados que podem apoiar a tomada de decisão.

## Importações Necessárias & Configs. Básicas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Configurações para exibir todas as colunas e linhas dos DataFrames

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

In [ ]:
# Carregando o dataset limpo

data_clean = pd.read_csv('../data/data_clean.csv')
data_clean['Data'] = pd.to_datetime(data_clean['Data'])
data_clean = data_clean.set_index('Data').sort_index(ascending=True)

## Análises Iniciais do Dataset

In [ ]:
# Visualização inicial dos dados

data_clean.head()

In [ ]:
data_clean.tail()

In [ ]:
# Informações sobre o DataFrame

print("Dataset Shape:", data_clean.shape)
print(data_clean.info())
print("\nDataset Description:\n", data_clean.describe())
print("\nMissing Values in Each Column:\n", data_clean.isnull().sum())


## Criando a coluna "Total_Venda"
- Total_Venda = Quantidade x Preço Unitário

In [ ]:
# Criando a coluna "Total_Venda", que é o produto da quantidade vendida pelo preço unitário

data_clean['Total_Venda'] = data_clean['Quantidade'] * data_clean['Preco_Unitario']

# Visualizando as primeiras linhas do DataFrame atualizado
data_clean.head()

### Agrupando as Colunas por "Total_Venda"

In [ ]:
# Groupby "Produto" e somando "Total_Venda" para cada produto

vendas_per_produto = pd.DataFrame(data_clean.groupby('Produto')['Total_Venda'].sum().sort_values(ascending=False))
print("Venda Total por Produto:")
display(vendas_per_produto)

Com base na tabela acima, podemos ver que o produto mais vendido é a Geladeira.

## Identificando Sazonalidades através do Gráfico de Linha

In [ ]:
# Vendas Mensais
vendas_mensais = data_clean.resample("ME")["Total_Venda"].sum()

plt.figure(figsize=(12, 6))
sns.lineplot(x=vendas_mensais.index, y=vendas_mensais.values, marker='o')
plt.title("Tendências de Vendas Mensais em 2023")
plt.xlabel("Mês")
plt.ylabel("Total de Vendas (R$)")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

Com base no gráfico acima, vemos que há uma clara sazonalidade, com picos e vales bem definidos ao longo do ano, que podem ser correlacionados a eventos do calendário e ao comportamento do mercado brasileiro.

#### Períodos de Alta:
- Pico de Final de Ano (Dez/Jan): O ponto mais alto de vendas ocorre em Janeiro, culminando o período de maior consumo do ano. Este pico é impulsionado pelas festas de fim de ano, recebimento do 13º salário e as promoções que se estendem desde a Black Friday.
- Pico de Meio de Ano (Junho): Um segundo pico significativo é observado em Junho. Este aumento é atribuído a datas como o dia dos namorados e as festas juninas, além da possível antecipação de compras para as férias escolares de Julho.
- Pico Pós-Carnaval (Março): Após uma baixa, as vendas se recuperam em Março. Este pode ser um reflexo do início do ano para o varejo após o Carnaval. Nesse mês, também há o dia do consumidor e, normalmente, são lançadas novas coleções.

#### Períodos de Baixa:
- Baixas Pós-Festas (Fevereiro): O mês de Fevereiro registra uma das quedas mais baixas. Isso se explica por ser um mês mais curto, pela concentração de gastos no Carnaval (muitas vezes somente em serviços) e por um período de reestruturação financeira após as despesas de fim de ano.
- Baixas Pós-Férias (Setembro): Uma forte queda nas vendas ocorre entre Agosto e Setembro. A principal hipótese é o fim do período de férias escolares, levando a uma redução nos gastos com lazer e a uma recomposição financeira para o restante do ano.

In [ ]:
# Plotando com a média móvel de 3 meses para suavizar a série temporal 
# e identificar tendências com maior facilidade.

plt.figure(figsize=(12, 6))
sns.lineplot(x=vendas_mensais.index, y=vendas_mensais.values, marker="o", label="Vendas Mensais")

media_movel = vendas_mensais.rolling(window=3).mean()
sns.lineplot(x=media_movel.index, y=media_movel.values, color="red", linewidth=2, label="Média Móvel (3 meses)")

plt.title("Tendências de Vendas Mensais em 2023")
plt.xlabel("Mês")
plt.ylabel("Total de Vendas (R$)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()

Como não ficou muito claro somente com as linhas do gráfico, inclui uma média móvel, que é capaz de suavizar flutuações pontuais e trazer mais insights para a análise, mostrando uma tendência mais clara de faturamento ao longo do ano.

#### Tendências de Crescimento:
- Apesar da queda em Abril, a média móvel projeta uma trajetória ascendente até Julho. Apesar da volatilidade mês a mês, a tendência é de crescimento no primeiro semestre.
- Em Outubro, há uma queda da média móvel, que é reflexo das baixas do início do segundo semestre. No entanto, a partir desse mês, a média móvel inicia uma trajetória de subida, que é em resposta ao fim de ano, dado que há um claro aumento de vendas nesse período, como mencionado anteriormente.